# Generate namelist input for simulated observations at ObsPack locations

This notebook reads ObsPack data and generates namelist input for CAM to produce output at fixed locations. The CAM namelist is documented [here](https://www.cesm.ucar.edu/models/cesm2/settings/2.2.0/cam_nml.html).


For surface station observation, we make use of the `fincl1lonlat` mechanism, which outputs column data at specific points at a specified frequency. The namelist documentation for `fincl1lonlat` is as follows:
> List of columns or contiguous columns at which the fincl1 fields will be
output. Individual columns are specified as a string using a longitude
degree (greater or equal to 0.) followed by a single character
(e)ast/(w)est identifer, an underscore '_' , and a latitude degree followed
by a single character (n)orth/(s)outh identifier.  For example, '10e_20n'
would pick the model column closest to 10 degrees east longitude by 20
degrees north latitude.  A group of contiguous columns can be specified
using bounding latitudes and longitudes separated by a colon.  For example,
'10e:20e_15n:20n' would select the model columns which fall with in the
longitude range from 10 east to 20 east and the latitude range from 15
north to 20 north.
Default: none


For the aircraft data, we make use of the `sathist` mechanism, which are set in the `satellite_options_nl` namelist. Here are the parameters:

`sathist_fincl`
> List of history fields to output along the satellite track specified by  sathist_track_infile
Default: none

`sathist_hfilename_spec`
> Satellite track history filename specifier.  See hfilename_spec
Default:  "%c.cam2.sat.%y-%m-%d-%s.nc"

`sathist_mfilt`
> Sets the maximum number of observation columns written to the satellite track history file
series.
Default: 100000

`sathist_nclosest`
> Sets the number of columns closest to the observation that should be output. Setting
this to a number greater than 1 allows for spatial interpolation in the post processing.
Default: 1

`sathist_ntimestep`
> Sets the number of timesteps closest to the observation that should be output. Setting
this to a number greater than 1 allows for temporal interpolation in the post processing.
Default: 1

`sathist_track_infile`
> Full pathname of the satellite track data used by the satellite track history
output feature.
Default: none

In [1]:
%load_ext autoreload
%autoreload 2

In [23]:
import os
import yaml

from datetime import datetime

import numpy as np
import xarray as xr

import cesm_tools

## Output directory

In [3]:
USER = os.environ['USER']
dataout = f"/glade/work/{USER}/cam-sd-cases/data"
os.makedirs(dataout, exist_ok=True)

obspack_dir = "/glade/work/mclong/sno-analysis/obspack_co2_1_GLOBALVIEWplus_v7.0_2021-08-18/data/nc"

## Produce `sathist` controls for aircraft observations

Example file:
/glade/work/cmcclus/field/satellite_profilelist_orcas_to_socrates_c190208.nc

In [25]:
%%time

campaign_keys = ["hip", "orc", "tom", "aoa"]

aircraft_files = {
    'hip': f"{obspack_dir}/co2_hip_aircraft-insitu_59_allvalid.nc",
    'orc': f"{obspack_dir}/co2_orc_aircraft-insitu_3_allvalid-merge10.nc",
    'tom': f"{obspack_dir}/co2_tom_aircraft-insitu_1_allvalid.nc",
    'aoa': f"{obspack_dir}/co2_aoa_aircraft-flask_19_allvalid.nc",
}
assert set(campaign_keys) == set(aircraft_files.keys())

ds_list = []
for n, key in enumerate(campaign_keys):
    with xr.open_dataset(aircraft_files[key]) as dsi:
        dsi = dsi.load()
        
    year = dsi.time_components[:, 0].astype(np.int32)
    month = dsi.time_components[:, 1].astype(np.int32)
    day = dsi.time_components[:, 2].astype(np.int32)
    hour = dsi.time_components[:, 3].astype(np.int32)
    minute = dsi.time_components[:, 4].astype(np.int32)
    second = dsi.time_components[:, 5].astype(np.int32)

    time_of_day = hour * 3600 + minute * 60 + second

    doy = xr.DataArray(
        [datetime(y, m, d).timetuple().tm_yday for y, m, d in zip(year, month, day)],
        dims=("obs"),
    )
    
    instr_num = xr.DataArray(
        (np.ones((dsi.sizes['obs'])) * n).astype(np.int32),
        dims=('obs'),
    )
    
    orbit_num = xr.DataArray(
        (np.ones((dsi.sizes['obs'])) * n).astype(np.int32), 
        dims=('obs'),
    )
    prof_num = xr.DataArray(
        np.arange(0, dsi.sizes['obs']), 
        dims=('obs'),
    )
    dso = xr.Dataset(
        {
            "julian": dsi.time,
            "date": year * 10000 + month * 100 + day,
            "lat": dsi.latitude,
            "lon": dsi.longitude,
            "time": time_of_day,
            "doy": doy,
            "instr_num": instr_num,
            "orbit_num": orbit_num,
            "prof_num": prof_num,
        }
    )

    dso.time.encoding['dtype'] = np.int32 
    dso.julian.encoding['dtype'] = np.float64
    dso.julian.encoding["units"] = "days since 2000-01-01 00:00:00"
    dso.julian.attrs["long_name"] = "UTC Time"
        
    dso.time.attrs["units"] = "s"
    dso.time.attrs["long_name"] = "time of day"

    dso.date.attrs["units"] = "yyyymmdd"
    dso.date.attrs["long_name"] = "date[yyyymmdd]"

    dso.instr_num.encoding['dtype'] = np.int32
    dso.prof_num.encoding['dtype'] = np.int32
    dso.orbit_num.encoding['dtype'] = np.int32
    
    dso = dso.rename({"obs": "profs"})
    
    ds_list.append(dso)

dso = xr.concat(ds_list, dim='profs')

dso.orbit_num.attrs['long_name'] = 'ObsPack File: ' + '; '.join([f"{n} = {os.path.basename(aircraft_files[key])}" for n, key in enumerate(campaign_keys)])
dso.prof_num.attrs['long_name'] = 'index into ObsPack file: 0-(N-1)'

for v in dso.variables:
    if 'comment' in dso[v].attrs:
        del dso[v].attrs['comment']

dso

CPU times: user 33.1 s, sys: 40.1 ms, total: 33.1 s
Wall time: 33.2 s


<xarray.Dataset>
Dimensions:    (profs: 320637)
Dimensions without coordinates: profs
Data variables:
    julian     (profs) datetime64[ns] 2009-01-09T20:28:45 ... 2020-12-10T05:3...
    date       (profs) int32 20090109 20090109 20090109 ... 20201210 20201210
    lat        (profs) float32 45.98 45.99 46.0 46.01 ... 31.87 32.81 33.75
    lon        (profs) float32 -108.9 -108.9 -108.9 -108.9 ... 139.9 140.0 140.3
    time       (profs) int32 73725 73735 73745 73755 ... 18090 18690 19410 20070
    doy        (profs) int64 9 9 9 9 9 9 9 9 ... 344 344 344 345 345 345 345 345
    instr_num  (profs) int32 0 0 0 0 0 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3 3 3 3 3 3
    orbit_num  (profs) int32 0 0 0 0 0 0 0 0 0 0 0 0 ... 3 3 3 3 3 3 3 3 3 3 3 3
    prof_num   (profs) int64 0 1 2 3 4 5 6 ... 2738 2739 2740 2741 2742 2743

In [26]:
ndx = np.argsort(dso.julian)
datestamp = datetime.now().strftime("%Y%m%d")

file_out = f"{dataout}/sathist_infile.{'-'.join(campaign_keys)}.{datestamp}.nc"
cesm_tools.to_netcdf_clean(dso.isel(profs=ndx), file_out)
cesm_tools.ncks_fl_fmt64bit(file_out)

------------------------------
Writing /glade/work/mclong/cam-sd-cases/data/sathist_infile.hip-orc-tom-aoa.20220303.nc
netcdf sathist_infile.hip-orc-tom-aoa.20220303 {
dimensions:
	profs = 320637 ;
variables:
	double julian(profs) ;
		julian:_FillValue = 9.96920996838687e+36 ;
		julian:long_name = "UTC Time" ;
		julian:units = "days since 2000-01-01" ;
		julian:calendar = "proleptic_gregorian" ;
	int date(profs) ;
		date:_FillValue = -2147483647 ;
		date:units = "yyyymmdd" ;
		date:long_name = "date[yyyymmdd]" ;
	float lat(profs) ;
		lat:_FillValue = 9.96921e+36f ;
		lat:standard_name = "latitude" ;
		lat:long_name = "sample_latitude_in_decimal_degrees" ;
		lat:units = "degrees_north" ;
	float lon(profs) ;
		lon:_FillValue = 9.96921e+36f ;
		lon:standard_name = "longitude" ;
		lon:long_name = "sample_longitude_in_decimal_degrees" ;
		lon:units = "degrees_east" ;
	int time(profs) ;
		time:_FillValue = -2147483647 ;
		time:units = "s" ;
		time:long_name = "time of day" ;
	int doy(profs) 

## Configure output for station locations

In [24]:
surface_files = [
    f"{obspack_dir}/co2_alt_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_brw_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_cba_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_cgo_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_kum_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_ljo_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_mlo_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_psa_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_smo_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_spo_surface-flask_4_representative.nc",
    f"{obspack_dir}/co2_gould_shipboard-insitu_1_allvalid.nc",
]

dsets_stn = {}
for f in surface_files:
    stn = os.path.basename(f).split('_')[1]
    with xr.open_dataset(f) as ds:
        dsets_stn[stn] = ds.load()        



def lonlat_to_str(lon, lat):
    lat_id = 's' if lat < 0 else 'n'
    lon_id = 'w' if lon < 0 else 'e'
    return f'{np.abs(lon):0.3f}{lon_id}_{np.abs(lat):0.3f}{lat_id}'
    
gould_lat_bin_e = np.arange(-66, -54, 2)
gould_lat_bin_c = np.vstack((gould_lat_bin_e[:-1], gould_lat_bin_e[1:])).mean(axis=0)

fincl1lonlat = {}
for key, ds in dsets_stn.items():
    if key == 'gould':        
        ndx = np.where(ds.latitude < -40)[0]
        dsi = ds.isel(obs=ndx)
        dsi = dsi.groupby_bins(dsi.latitude, gould_lat_bin_e).mean()
        for i, (lon, lat) in enumerate(zip(dsi.longitude.values, dsi.latitude.values)):
            fincl1lonlat[f'{key}_{np.abs(gould_lat_bin_c[i]):0.0f}S'] = lonlat_to_str(lon, lat)
    else:
        assert ds.latitude.std() < 0.01 and ds.longitude.std() < 0.01, key
        fincl1lonlat[key] = lonlat_to_str(ds.longitude.mean().values, ds.latitude.mean().values)

        
with open('ObsPack-lonlat-fincl-specifier.yml', 'w') as fid:
    yaml.dump(fincl1lonlat, fid)
fincl1lonlat        

{'alt': '62.507w_82.451n',
 'brw': '156.611w_71.323n',
 'cba': '162.720w_55.210n',
 'cgo': '144.690e_40.683s',
 'kum': '154.888w_19.561n',
 'ljo': '117.257w_32.867n',
 'mlo': '155.576w_19.536n',
 'psa': '64.053w_64.774s',
 'smo': '170.564w_14.247s',
 'spo': '24.800w_89.980s',
 'gould_65S': '63.855w_64.785s',
 'gould_63S': '61.123w_63.077s',
 'gould_61S': '60.621w_61.042s',
 'gould_59S': '63.317w_59.026s',
 'gould_57S': '64.222w_57.023s'}